In [104]:
import sys, os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from pytrends.request import TrendReq 
pytrend = TrendReq()


# Google Trends

## Popular Queries 

The top 50 queries related to the words coronavirus and covid-19

In [105]:
# words to search for
keywords = ['coronavirus','covid-19']

# Obtaining the payload
pytrend.build_payload(keywords, timeframe='2020-01-01 2020-03-20')

# Obtaining related queries
related = pytrend.related_queries()
searches = related[keywords[0]]['top']['query'].append(related[keywords[1]]['top']['query'])
searches = searches.reset_index().drop(columns=["index"])['query']
print(searches)
searches = searches.to_list()
searches.extend(keywords)

0                      corona
1          coronavirus update
2        coronavirus symptoms
3            news coronavirus
4           coronavirus cases
5                corona virus
6              coronavirus uk
7           china coronavirus
8             coronavirus map
9          coronavirus italia
10             el coronavirus
11         coronavirus france
12       coronavirus sintomas
13          india coronavirus
14             coronavirus us
15    symptoms of coronavirus
16          coronavirus italy
17            coronavirus usa
18            usa coronavirus
19         coronavirus españa
20           coronavirus tips
21          coronavirus death
22           coronavirus live
23        what is coronavirus
24         coronavirus latest
25       covid-19 coronavirus
26                      covid
27             covid-19 cases
28                   covid 19
29             covid-19 virus
30               covid-19 map
31          covid-19 symptoms
32              covid-19 news
33        

## Interest by Region
Retrieving interest by region data saved to a csv file in the data directory

In [107]:
# only allowed 5 words max in a list
pytrend.build_payload(kw_list, timeframe='2020-01-01 2020-03-20')

# Interest by Region
# results are relative search volumes
# standardizing it from 0 to 100, where 100 represents the highest share of the term over a time series
# standardized on each request - data isnt really that useful for our requirements
df = pytrend.interest_by_region(inc_low_vol=True)
print(df)



                Coronavirus  covid-19
geoName                              
Afghanistan              97         3
Albania                  99         1
Algeria                  98         2
American Samoa           99         1
Andorra                 100         0
...                     ...       ...
Western Sahara           96         4
Yemen                    92         8
Zambia                   98         2
Zimbabwe                 98         2
Åland Islands            96         4

[250 rows x 2 columns]


## Historical Interest


In [ ]:

df = pytrend.get_historical_interest(kw_list, year_start=2020, month_start=1, day_start=1, hour_start=0, year_end=2020, month_end=3, day_end=16, hour_end=0, cat=0, geo='', gprop='', sleep=0)
print(df.head(10))
df.reset_index().plot(x='date', y=kw_list, figsize=(200, 50), kind ='line')

In [ ]:

df = pytrend.get_historical_interest(kw_list, year_start=2020, month_start=1, day_start=1, hour_start=0, year_end=2020, month_end=3, day_end=16, hour_end=0, cat=0, geo='', gprop='', sleep=0)
df['Date'] = df.reset_index()['date'].dt.date
df['Time'] = df.reset_index()['date'].dt.time
df.set_index('Date',drop=True)



The request failed: Google returned a response with code 500.
